In [1]:
from abc import ABC, abstractmethod
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3 import A2C as sb_A2C
from stable_baselines3 import PPO as sb_PPO
from stable_baselines3 import DDPG as sb_DDPG
from stable_baselines3 import TD3 as sb_TD3
from stable_baselines3.common.noise import ActionNoise
from stable_baselines3.common.buffers import ReplayBuffer
from stable_baselines3.common.type_aliases import Schedule
from gym import spaces
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch as th
import yaml
import gym
from datetime import datetime
from sys import path
from os.path import dirname as dir
from typing import Any, Dict, Optional, Tuple, Type, Union
path.append(dir(path[0]))
print(path)
#__package__ = "examples"

from FinancialDataLayer.DataCollection.DataDownloader import DataDownloader
from FinancialDataLayer.DataProcessing.DefaultFeatureEngineer import DefaultFeatureEngineer
from AgentLayer.DataSplitter.TimeSeriesSplitter import TimeSeriesSplitter

['/Users/doganparlak/Desktop/Master_2.2/Master_Project/uniFi_github/uniFi/AgentLayer', '/usr/local/Cellar/python@3.8/3.8.8_1/Frameworks/Python.framework/Versions/3.8/lib/python38.zip', '/usr/local/Cellar/python@3.8/3.8.8_1/Frameworks/Python.framework/Versions/3.8/lib/python3.8', '/usr/local/Cellar/python@3.8/3.8.8_1/Frameworks/Python.framework/Versions/3.8/lib/python3.8/lib-dynload', '', '/Users/doganparlak/Library/Python/3.8/lib/python/site-packages', '/usr/local/lib/python3.8/site-packages', '/usr/local/lib/python3.8/site-packages/selenium-3.141.0-py3.8.egg', '/usr/local/lib/python3.8/site-packages/urllib3-1.26.4-py3.8.egg', '/Users/doganparlak/Desktop/Master_2.2/Master_Project/uniFi_github/uniFi']


In [16]:
import pickle
import pypfopt
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt import objective_functions
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor

In [3]:
class Environment(gym.Env, ABC):

    @abstractmethod
    def reset(self):
        pass

    @abstractmethod
    def step(self, action):
        pass

    @abstractmethod
    def render(self, mode="human"):
        pass

    @abstractmethod
    def get_env(self):
        pass

    @staticmethod
    def softmax_normalization(actions):
        numerator = np.exp(actions)
        denominator = np.sum(np.exp(actions))
        softmax_output = numerator / denominator
        return softmax_output


In [3]:
class Agent(ABC):

    @abstractmethod
    def train_model():
        pass

    @abstractmethod
    def predict():
        pass

    @abstractmethod
    def save_model():
        pass
    
    @abstractmethod
    def load_model():
        pass

In [1]:
class ConventionalAgent(Agent, ABC):

    @abstractmethod
    def train_model():
        pass

    @abstractmethod
    def predict():
        pass

    @abstractmethod
    def save_model():
        pass

    @abstractmethod
    def load_model():
        pass
    
    @abstractmethod
    def _return_predict():
        pass

    @abstractmethod
    def _weight_optimization():
        pass

NameError: name 'Agent' is not defined

In [8]:
class RLAgent(Agent, ABC):

    @abstractmethod
    def train_model():
        pass

    @abstractmethod
    def predict():
        
        pass

    @abstractmethod
    def save_model():
        pass

    @abstractmethod
    def load_model():
        pass

In [19]:
class RFAgent(ConventionalAgent):

    def __init__(self,
                n_estimators = 100,
                criterion = "squared_error",
                max_depth = None,
                min_samples_split = 2,
                min_samples_leaf = 1,
                min_weight_fraction_leaf = 0,
                max_features = 1,
                max_leaf_nodes = None,
                min_impurity_decrease = 0,
                bootstrap = True,
                oob_score = False,
                n_jobs = None,
                random_state = None,
                verbose = 0,
                warm_start = False,
                ccp_alpha = 0,
                max_samples = None):

        self.model = RandomForestRegressor(n_estimators= n_estimators,
                            criterion= criterion,
                            max_depth=max_depth,
                            min_samples_split=min_samples_split,
                            min_samples_leaf=min_samples_leaf,
                            min_weight_fraction_leaf=min_weight_fraction_leaf,
                            max_features=max_features,
                            max_leaf_nodes= max_leaf_nodes,
                            min_impurity_decrease=min_impurity_decrease,
                            bootstrap=bootstrap,
                            oob_score=oob_score,
                            n_jobs=n_jobs,
                            random_state= random_state,
                            verbose=verbose,
                            warm_start=warm_start,
                            ccp_alpha=ccp_alpha,
                            max_samples=max_samples)
    
    def train_model(self, train_x, train_y, **train_params):
        '''
        *Trains the model*
        Input: Train data x and train data y
        Output: Linear Regression Model
        '''
        try:
            trained = self.model.fit(train_x, train_y.ravel(), **train_params)
            print("Model trained succesfully")
            return trained
        except Exception as e:
            print("training unsuccessful")    
    
    def predict(self,
                test_data, 
                initial_capital = 0,
                tech_indicator_list = [
                        "macd",
                        "boll_ub",
                        "boll_lb",
                        "rsi_30",
                        "cci_30",
                        "dx_30",
                        "close_30_sma",
                        "close_60_sma",
                    ]):

            meta_coefficient = {"date": [], "weights": []}
            unique_trade_date = test_data.date.unique()
            portfolio = pd.DataFrame(index=range(1), columns=unique_trade_date)
            portfolio.loc[0, unique_trade_date[0]] = initial_capital

            for i in range(len(unique_trade_date) - 1):
                mu, sigma, tics, df_current, df_next = self._return_predict(
                    unique_trade_date, test_data, i, tech_indicator_list)

                portfolio_value = self._weight_optimization(
                    i, unique_trade_date, meta_coefficient, mu, sigma, tics, portfolio, df_current, df_next)
        
            portfolio = portfolio_value
            portfolio = portfolio.T
            portfolio.columns = ['account_value']
            portfolio = portfolio.reset_index()
            portfolio.columns = ['date', 'account_value']

            '''Backtest hasn't been implemented yet, hence commented.'''
            #stats = backtest_stats(portfolio, value_col_name='account_value')
            
            portfolio_cumprod = (
                portfolio.account_value.pct_change()+1).cumprod()-1

            return portfolio, portfolio_cumprod, pd.DataFrame(meta_coefficient)
    
    def _return_predict(self, unique_trade_date, test_data, i, tech_indicator_list):

        current_date = unique_trade_date[i]
        next_date = unique_trade_date[i+1]

        df_current = test_data[test_data.date ==
                                  current_date].reset_index(drop=True)
        df_next = test_data[test_data.date ==
                               next_date].reset_index(drop=True)

        tics = df_current['tic'].values
        features = df_current[tech_indicator_list].values

        predicted_y = self.model.predict(features)
        mu = predicted_y
        sigma = risk_models.sample_cov(
            df_current.return_list[0], returns_data=True)

        return mu, sigma, tics, df_current, df_next
    
    def _weight_optimization(self, i, unique_trade_date, meta_coefficient, mu, sigma, tics, portfolio, df_current, df_next):

        current_date = unique_trade_date[i]
        predicted_y_df = pd.DataFrame(
            {"tic": tics.reshape(-1,), "predicted_y": mu.reshape(-1,)})
        min_weight, max_weight = 0, 1

        ef = EfficientFrontier(mu, sigma)
        weights = ef.nonconvex_objective(
            objective_functions.sharpe_ratio,
            objective_args=(ef.expected_returns, ef.cov_matrix),
            weights_sum_to_one=True,
            constraints=[
                # greater than min_weight
                {"type": "ineq", "fun": lambda w: w - min_weight},
                # less than max_weight
                {"type": "ineq", "fun": lambda w: max_weight - w},
            ],
        )

        weight_df = {"tic": [], "weight": []}
        meta_coefficient["date"] += [current_date]

        for item in weights:
            weight_df['tic'] += [item]
            weight_df['weight'] += [weights[item]]

        weight_df = pd.DataFrame(weight_df).merge(predicted_y_df, on=['tic'])
        meta_coefficient["weights"] += [weight_df]
        cap = portfolio.iloc[0, i]
        # current cash invested for each stock
        current_cash = [element * cap for element in list(weights.values())]
        # current held shares
        current_shares = list(np.array(current_cash) / np.array(df_current.close))
        # next time period price
        next_price = np.array(df_next.close)
        portfolio.iloc[0, i+1] = np.dot(current_shares, next_price)

        return portfolio 

    def save_model(self,  file_name):
        with open(file_name, 'wb') as files:
            pickle.dump(self.model, files)
        print("Model saved succesfully.")


    def load_model(self, file_name):
        with open(file_name, 'rb') as f:
            self.model = pickle.load(f)
        print("Model loaded succesfully.")
        return self.model

In [21]:
class DTAgent(ConventionalAgent):
    
    def __init__(self,
                criterion = "squared_error",
                splitter = "best",
                max_depth = None,
                min_samples_split = 2,
                min_samples_leaf = 1,
                min_weight_fraction_leaf = 0,
                max_features = None,
                random_state = None,
                max_leaf_nodes = None,
                min_impurity_decrease = 0,
                ccp_alpha = 0):
                

        self.model = DecisionTreeRegressor( criterion = criterion,
                                            splitter = splitter,
                                            max_depth = max_depth,
                                            min_samples_split = min_samples_split,
                                            min_samples_leaf = min_samples_leaf,
                                            min_weight_fraction_leaf = min_weight_fraction_leaf,
                                            max_features = max_features,
                                            random_state = random_state,
                                            max_leaf_nodes = max_leaf_nodes,
                                            min_impurity_decrease = min_impurity_decrease,
                                            ccp_alpha = ccp_alpha)
                                            
    def train_model(self, train_x, train_y, **train_params):
        '''
        *Trains the model*
        Input: Train data x and train data y
        Output: Linear Regression Model
        '''
        try:
            trained = self.model.fit(train_x, train_y, **train_params)
            print("Model trained succesfully")
            return trained
        except Exception as e:
            print("training unsuccessful")    
    
    def predict(self,
                test_data, 
                initial_capital = 0,
                tech_indicator_list = [
                        "macd",
                        "boll_ub",
                        "boll_lb",
                        "rsi_30",
                        "cci_30",
                        "dx_30",
                        "close_30_sma",
                        "close_60_sma",
                    ]):

            meta_coefficient = {"date": [], "weights": []}
            unique_trade_date = test_data.date.unique()
            portfolio = pd.DataFrame(index=range(1), columns=unique_trade_date)
            portfolio.loc[0, unique_trade_date[0]] = initial_capital

            for i in range(len(unique_trade_date) - 1):
                mu, sigma, tics, df_current, df_next = self._return_predict(
                    unique_trade_date, test_data, i, tech_indicator_list)

                portfolio_value = self._weight_optimization(
                    i, unique_trade_date, meta_coefficient, mu, sigma, tics, portfolio, df_current, df_next)
        
            portfolio = portfolio_value
            portfolio = portfolio.T
            portfolio.columns = ['account_value']
            portfolio = portfolio.reset_index()
            portfolio.columns = ['date', 'account_value']

            '''Backtest hasn't been implemented yet, hence commented.'''
            #stats = backtest_stats(portfolio, value_col_name='account_value')
            
            portfolio_cumprod = (
                portfolio.account_value.pct_change()+1).cumprod()-1

            return portfolio, portfolio_cumprod, pd.DataFrame(meta_coefficient)
            
    def _return_predict(self, unique_trade_date, test_data, i, tech_indicator_list):

        current_date = unique_trade_date[i]
        next_date = unique_trade_date[i+1]

        df_current = test_data[test_data.date ==
                                  current_date].reset_index(drop=True)
        df_next = test_data[test_data.date ==
                               next_date].reset_index(drop=True)

        tics = df_current['tic'].values
        features = df_current[tech_indicator_list].values

        predicted_y = self.model.predict(features)
        mu = predicted_y
        sigma = risk_models.sample_cov(
            df_current.return_list[0], returns_data=True)

        return mu, sigma, tics, df_current, df_next

    def _weight_optimization(self, i, unique_trade_date, meta_coefficient, mu, sigma, tics, portfolio, df_current, df_next):

        current_date = unique_trade_date[i]
        predicted_y_df = pd.DataFrame(
            {"tic": tics.reshape(-1,), "predicted_y": mu.reshape(-1,)})
        min_weight, max_weight = 0, 1

        ef = EfficientFrontier(mu, sigma)
        weights = ef.nonconvex_objective(
            objective_functions.sharpe_ratio,
            objective_args=(ef.expected_returns, ef.cov_matrix),
            weights_sum_to_one=True,
            constraints=[
                # greater than min_weight
                {"type": "ineq", "fun": lambda w: w - min_weight},
                # less than max_weight
                {"type": "ineq", "fun": lambda w: max_weight - w},
            ],
        )

        weight_df = {"tic": [], "weight": []}
        meta_coefficient["date"] += [current_date]

        for item in weights:
            weight_df['tic'] += [item]
            weight_df['weight'] += [weights[item]]

        weight_df = pd.DataFrame(weight_df).merge(predicted_y_df, on=['tic'])
        meta_coefficient["weights"] += [weight_df]
        cap = portfolio.iloc[0, i]
        # current cash invested for each stock
        current_cash = [element * cap for element in list(weights.values())]
        # current held shares
        current_shares = list(np.array(current_cash) / np.array(df_current.close))
        # next time period price
        next_price = np.array(df_next.close)
        portfolio.iloc[0, i+1] = np.dot(current_shares, next_price)

        return portfolio 
    
    def save_model(self,  file_name):
        with open(file_name, 'wb') as files:
            pickle.dump(self.model, files)
        print("Model saved succesfully.")


    def load_model(self, file_name):
        with open(file_name, 'rb') as f:
            self.model = pickle.load(f)
        print("Model loaded succesfully.")
        return self.model

In [6]:
class SVRAgent(ConventionalAgent):
    
    def __init__(self,
                kernel = 'rbf',
                degree = 3,
                gamma = 'scale',
                coef0 = 0,
                tol = 0.001,
                C = 1,
                epsilon = 0.1,
                shrinking = True,
                cache_size = 200,
                verbose = False,
                max_iter = -1):  

        self.model = SVR(kernel= kernel,
                        degree= degree,
                        gamma= gamma, 
                        coef0= coef0,
                        tol= tol,
                        C= C,
                        epsilon= epsilon,
                        shrinking= shrinking,
                        cache_size= cache_size,
                        verbose= verbose,
                        max_iter= max_iter)

    def train_model(self, train_x, train_y, **train_params):
        '''
        *Trains the model*
        Input: Train data x and train data y
        Output: Linear Regression Model
        '''
        try:
            trained = self.model.fit(train_x, train_y.ravel(), **train_params)
            print("Model trained succesfully")
            return trained
        except Exception as e:
            print("training unsuccessful")

    def predict(self,
               test_data, 
               initial_capital = 0,
               tech_indicator_list = [
                    "macd",
                    "boll_ub",
                    "boll_lb",
                    "rsi_30",
                    "cci_30",
                    "dx_30",
                    "close_30_sma",
                    "close_60_sma",
                ]):

        meta_coefficient = {"date": [], "weights": []}
        unique_trade_date = test_data.date.unique()
        portfolio = pd.DataFrame(index=range(1), columns=unique_trade_date)
        portfolio.loc[0, unique_trade_date[0]] = initial_capital

        for i in range(len(unique_trade_date) - 1):
            mu, sigma, tics, df_current, df_next = self._return_predict(
                unique_trade_date, test_data, i, tech_indicator_list)

            portfolio_value = self._weight_optimization(
                i, unique_trade_date, meta_coefficient, mu, sigma, tics, portfolio, df_current, df_next)
    
        portfolio = portfolio_value
        portfolio = portfolio.T
        portfolio.columns = ['account_value']
        portfolio = portfolio.reset_index()
        portfolio.columns = ['date', 'account_value']

        '''Backtest hasn't been implemented yet, hence commented.'''
        #stats = backtest_stats(portfolio, value_col_name='account_value')
        
        portfolio_cumprod = (
            portfolio.account_value.pct_change()+1).cumprod()-1

        return portfolio, portfolio_cumprod, pd.DataFrame(meta_coefficient)
    
    def _return_predict(self, unique_trade_date, test_data, i, tech_indicator_list):

        current_date = unique_trade_date[i]
        next_date = unique_trade_date[i+1]

        df_current = test_data[test_data.date ==
                                  current_date].reset_index(drop=True)
        df_next = test_data[test_data.date ==
                               next_date].reset_index(drop=True)

        tics = df_current['tic'].values
        features = df_current[tech_indicator_list].values

        predicted_y = self.model.predict(features)
        mu = predicted_y
        sigma = risk_models.sample_cov(
            df_current.return_list[0], returns_data=True)

        return mu, sigma, tics, df_current, df_next
    
    def _weight_optimization(self, i, unique_trade_date, meta_coefficient, mu, sigma, tics, portfolio, df_current, df_next):

        current_date = unique_trade_date[i]
        predicted_y_df = pd.DataFrame(
            {"tic": tics.reshape(-1,), "predicted_y": mu.reshape(-1,)})
        min_weight, max_weight = 0, 1

        ef = EfficientFrontier(mu, sigma)
        weights = ef.nonconvex_objective(
            objective_functions.sharpe_ratio,
            objective_args=(ef.expected_returns, ef.cov_matrix),
            weights_sum_to_one=True,
            constraints=[
                # greater than min_weight
                {"type": "ineq", "fun": lambda w: w - min_weight},
                # less than max_weight
                {"type": "ineq", "fun": lambda w: max_weight - w},
            ],
        )

        weight_df = {"tic": [], "weight": []}
        meta_coefficient["date"] += [current_date]

        for item in weights:
            weight_df['tic'] += [item]
            weight_df['weight'] += [weights[item]]

        weight_df = pd.DataFrame(weight_df).merge(predicted_y_df, on=['tic'])
        meta_coefficient["weights"] += [weight_df]
        cap = portfolio.iloc[0, i]
        # current cash invested for each stock
        current_cash = [element * cap for element in list(weights.values())]
        # current held shares
        current_shares = list(np.array(current_cash) / np.array(df_current.close))
        # next time period price
        next_price = np.array(df_next.close)
        portfolio.iloc[0, i+1] = np.dot(current_shares, next_price)

        return portfolio 
    
    def save_model(self,  file_name):
        with open(file_name, 'wb') as files:
            pickle.dump(self.model, files)
        print("Model saved succesfully.")


    def load_model(self, file_name):
        with open(file_name, 'rb') as f:
            self.model = pickle.load(f)
        print("Model loaded succesfully.")
        return self.model

In [66]:
class LRAgent(ConventionalAgent):

   def __init__(self,
                fit_intercept = True,
                copy_X = True,
                positive = False):  

        self.model = LinearRegression(fit_intercept=fit_intercept,
                                      copy_X= copy_X,
                                      positive= positive)

   def train_model(self, train_x, train_y, **train_params):
        '''
        *Trains the model*
        Input: Train data x and train data y
        Output: Linear Regression Model
        '''
        try:
            trained_reg = self.model.fit(train_x, train_y, **train_params)
            print("Model trained succesfully")
            return trained_reg
        except Exception as e:
            print("training unsuccessful")
    
   def predict(self,
               test_data, 
               initial_capital = 0,
               tech_indicator_list = [
                    "macd",
                    "boll_ub",
                    "boll_lb",
                    "rsi_30",
                    "cci_30",
                    "dx_30",
                    "close_30_sma",
                    "close_60_sma",
                ]):

        meta_coefficient = {"date": [], "weights": []}
        unique_trade_date = test_data.date.unique()
        portfolio = pd.DataFrame(index=range(1), columns=unique_trade_date)
        portfolio.loc[0, unique_trade_date[0]] = initial_capital

        for i in range(len(unique_trade_date) - 1):
            mu, sigma, tics, df_current, df_next = self._return_predict(
                unique_trade_date, test_data, i, tech_indicator_list)

            portfolio_value = self._weight_optimization(
                i, unique_trade_date, meta_coefficient, mu, sigma, tics, portfolio, df_current, df_next)
    
        portfolio = portfolio_value
        portfolio = portfolio.T
        portfolio.columns = ['account_value']
        portfolio = portfolio.reset_index()
        portfolio.columns = ['date', 'account_value']

        '''Backtest hasn't been implemented yet, hence commented.'''
        #stats = backtest_stats(portfolio, value_col_name='account_value')
        
        portfolio_cumprod = (
            portfolio.account_value.pct_change()+1).cumprod()-1

        return portfolio, portfolio_cumprod, pd.DataFrame(meta_coefficient)

   def _return_predict(self, unique_trade_date, test_data, i, tech_indicator_list):

        current_date = unique_trade_date[i]
        next_date = unique_trade_date[i+1]

        df_current = test_data[test_data.date ==
                                  current_date].reset_index(drop=True)
        df_next = test_data[test_data.date ==
                               next_date].reset_index(drop=True)

        tics = df_current['tic'].values
        features = df_current[tech_indicator_list].values

        predicted_y = self.model.predict(features)
        mu = predicted_y
        sigma = risk_models.sample_cov(
            df_current.return_list[0], returns_data=True)

        return mu, sigma, tics, df_current, df_next

   def _weight_optimization(self, i, unique_trade_date, meta_coefficient, mu, sigma, tics, portfolio, df_current, df_next):

        current_date = unique_trade_date[i]
        predicted_y_df = pd.DataFrame(
            {"tic": tics.reshape(-1,), "predicted_y": mu.reshape(-1,)})
        min_weight, max_weight = 0, 1

        ef = EfficientFrontier(mu, sigma)
        weights = ef.nonconvex_objective(
            objective_functions.sharpe_ratio,
            objective_args=(ef.expected_returns, ef.cov_matrix),
            weights_sum_to_one=True,
            constraints=[
                # greater than min_weight
                {"type": "ineq", "fun": lambda w: w - min_weight},
                # less than max_weight
                {"type": "ineq", "fun": lambda w: max_weight - w},
            ],
        )

        weight_df = {"tic": [], "weight": []}
        meta_coefficient["date"] += [current_date]

        for item in weights:
            weight_df['tic'] += [item]
            weight_df['weight'] += [weights[item]]

        weight_df = pd.DataFrame(weight_df).merge(predicted_y_df, on=['tic'])
        meta_coefficient["weights"] += [weight_df]
        cap = portfolio.iloc[0, i]
        # current cash invested for each stock
        current_cash = [element * cap for element in list(weights.values())]
        # current held shares
        current_shares = list(np.array(current_cash) / np.array(df_current.close))
        # next time period price
        next_price = np.array(df_next.close)
        portfolio.iloc[0, i+1] = np.dot(current_shares, next_price)

        return portfolio 

   def save_model(self,  file_name):
        with open(file_name, 'wb') as files:
            pickle.dump(self.model, files)
        print("Model saved succesfully.")


   def load_model(self, file_name):
        with open(file_name, 'rb') as f:
            self.model = pickle.load(f)
        print("Model loaded succesfully.")
        return self.model


In [10]:
class PortfolioEnv(Environment):

    def __init__(self,
                 df: pd.DataFrame,  # input data
                 stock_dim: int,  # number of unique securities in the investment universe
                 hmax: float,  # maximum number of shares to trade
                 initial_amount: float,  # initial cash value
                 transaction_cost_pct: float,  # transaction cost percentage per trade
                 reward_scaling: float,  # scaling factor for reward as training progresses
                 state_space: int,  # the dimension of input features (state space)
                 action_space: int,  # number of actions, which is equal to portfolio dimension
                 tech_indicator_list: list,  # a list of technical indicator names
                 turbulence_threshold=None,  # a threshold to control risk aversion
                 lookback=252,  #
                 day=0):  # an increment number to control date

        self.df = df
        self.day = day
        self.lookback = lookback
        self.stock_dim = stock_dim
        self.hmax = hmax
        self.initial_amount = initial_amount
        self.transaction_cost_pct = transaction_cost_pct
        self.reward_scaling = reward_scaling
        self.state_space = state_space
        self.action_space = action_space
        self.tech_indicator_list = tech_indicator_list

        # action_space normalization and shape is self.stock_dim
        self.action_space = spaces.Box(low=0, high=1, shape=(self.action_space,))
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf,
                                            shape=(self.state_space + len(self.tech_indicator_list), self.state_space))
        

        # load data from a pandas dataframe
        
        ##FINRL APPROACH
        #self.df.set_index("date", drop = True, inplace=True)
        
        self.data = self.df.loc[self.day,:]
        self.covs = self.data['cov_list'].values[0]
        self.state =  np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list ], axis=0)
        self.terminal = False
        #self.turbulence_threshold = turbulence_threshold
        # initalize state: initial portfolio return + individual stock return + individual weights
        self.portfolio_value = self.initial_amount

        # memorize portfolio value each step
        self.asset_memory = [self.initial_amount]
        # memorize portfolio return each step
        self.portfolio_return_memory = [0]
        self.actions_memory = [[1 / self.stock_dim] * self.stock_dim]
        self.date_memory=[self.data.date.unique()[0]]

    def reset(self):
        self.asset_memory = [self.initial_amount]
        self.day = 0
        self.data = self.df.loc[self.day,:]
        # load states
        self.covs = self.data['cov_list'].values[0]
        self.state =  np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list ], axis=0)
                
        self.portfolio_value = self.initial_amount
        #self.cost = 0
        #self.trades = 0
        self.terminal = False 
        self.portfolio_return_memory = [0]
        self.actions_memory=[[1/self.stock_dim] * self.stock_dim]
        self.date_memory=[self.data.date.unique()[0]] 
        return self.state

    def step(self, actions):
        self.terminal = self.day >= len(self.df.index.unique()) - 1
        if self.terminal:
            df = pd.DataFrame(self.portfolio_return_memory)
            print(df)
            df.columns = ['daily_return']
            plt.plot(df.daily_return.cumsum(), 'r')
           # plt.savefig('results/cumulative_reward.png')
            plt.close()

            plt.plot(self.portfolio_return_memory, 'r')
           #plt.savefig('results/rewards.png')
            plt.close()

            print("=================================")
            print("begin_total_asset:{}".format(self.asset_memory[0]))
            print("end_total_asset:{}".format(self.portfolio_value))

            df_daily_return = pd.DataFrame(self.portfolio_return_memory)
            df_daily_return.columns = ['daily_return']
            if df_daily_return['daily_return'].std() != 0:
                sharpe = (252 ** 0.5) * df_daily_return['daily_return'].mean() / \
                         df_daily_return['daily_return'].std()
                print("Sharpe: ", sharpe)
            print("=================================")

            return self.state, self.reward, self.terminal, {}

        else:
            weights = Environment.softmax_normalization(actions)
            self.actions_memory.append(weights)
            last_day_memory = self.data

            # load next state
            self.day += 1
            self.data = self.df.loc[self.day, :]
            self.covs = self.data['cov_list'].values[0]
            self.state =  np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list ], axis=0)         
            portfolio_return = sum(((self.data.close.values / last_day_memory.close.values) - 1) * weights)
            log_portfolio_return = np.log(sum((self.data.close.values / last_day_memory.close.values) * weights))
            # update portfolio value
            new_portfolio_value = self.portfolio_value * (1 + portfolio_return)
            self.portfolio_value = new_portfolio_value

            # save into memory
            self.portfolio_return_memory.append(portfolio_return)
            self.date_memory.append(self.data["date"].unique()[0])
            self.asset_memory.append(new_portfolio_value)

            # the reward is the new portfolio value or end portfolo value
            self.reward = new_portfolio_value

        return self.state, self.reward, self.terminal, {}

    def render(self, mode='human'):
        return self.state

    def save_asset_memory(self):
        date_list = self.date_memory
        portfolio_return = self.portfolio_return_memory
        # print(len(date_list))
        # print(len(asset_list))
        df_account_value = pd.DataFrame({'date': date_list, 'daily_return': portfolio_return})
        return df_account_value

    def save_action_memory(self):
        # date and close price length must match actions length
        date_list = self.date_memory
        df_date = pd.DataFrame(date_list)
        df_date.columns = ['date']

        action_list = self.actions_memory
        df_actions = pd.DataFrame(action_list)
        df_actions.columns = self.data.tic.values
        df_actions.index = df_date.date
        # df_actions = pd.DataFrame({'date':date_list,'actions':action_list})
        return df_actions

    def _seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

    def get_env(self):
        e = DummyVecEnv([lambda: self])
        obs = e.reset()
        return e, obs

In [7]:
class A2C(RLAgent):

    def __init__(self,
                 policy: "MlpPolicy",
                 env: None,
                 learning_rate: float = 7e-4,
                 n_steps: int = 5,
                 gamma: float = 0.99,
                 gae_lambda: float = 1.0,
                 ent_coef: float = 0.0,
                 vf_coef: float = 0.5,
                 max_grad_norm: float = 0.5,
                 rms_prop_eps: float = 1e-5,
                 use_rms_prop: bool = True,
                 use_sde: bool = False,
                 sde_sample_freq: int = -1,
                 normalize_advantage: bool = False,
                 tensorboard_log: Optional[str] = None,
                 create_eval_env: bool = False,
                 policy_kwargs: Optional[Dict[str, Any]] = None,
                 verbose: int = 0,
                 seed: Optional[int] = None,
                 device: Union[th.device, str] = "auto",
                 _init_setup_model: bool = True):

        self.env = env

        self.model = sb_A2C(policy = policy,
                            env=self.env,
                            learning_rate = learning_rate,
                            n_steps = n_steps,
                            gamma = gamma,
                            gae_lambda= gae_lambda,
                            ent_coef = ent_coef,
                            vf_coef = vf_coef,
                            max_grad_norm = max_grad_norm,
                            rms_prop_eps= rms_prop_eps,
                            use_rms_prop= use_rms_prop,
                            use_sde= use_sde,
                            sde_sample_freq= sde_sample_freq,
                            normalize_advantage= normalize_advantage,
                            tensorboard_log=tensorboard_log,  
                            create_eval_env= create_eval_env,
                            policy_kwargs=policy_kwargs,
                            verbose=verbose,
                            seed=seed,
                            device= device,
                            _init_setup_model = _init_setup_model)

    def train_model(self, **train_params):
        self.model = self.model.learn(**train_params)
        return self.model

    def predict(self, environment, **test_params):

        env_test, obs_test = environment.get_env()
        """make a prediction"""
        account_memory = []
        actions_memory = []

        env_test.reset()
        for i in range(len(environment.df.index.unique())):
            action, _states = self.model.predict(obs_test, **test_params)
            obs_test, rewards, dones, info = env_test.step(action)
            if i == (len(environment.df.index.unique()) - 2):
                account_memory = env_test.env_method(method_name="save_asset_memory")
                actions_memory = env_test.env_method(method_name="save_action_memory")
            if dones[0]:
                print("hit end!")
                break

        return account_memory[0], actions_memory[0]

    def save_model(self, path):
        self.model.save(path)

    def load_model(self, path):
        self.model = self.model.load(path)
        return self.model

In [8]:
class PPO(RLAgent):
    def __init__(self,
                policy: "MlpPolicy",
                env: None,
                learning_rate:  3e-4,
                n_steps: int = 2048,
                batch_size: int = 64,
                n_epochs: int = 10,
                gamma: float = 0.99,
                gae_lambda: float = 0.95,
                clip_range: Union[float, Schedule] = 0.2,
                clip_range_vf: Union[None, float, Schedule] = None,
                normalize_advantage: bool = True,
                ent_coef: float = 0.0,
                vf_coef: float = 0.5,
                max_grad_norm: float = 0.5,
                use_sde: bool = False,
                sde_sample_freq: int = -1,
                target_kl: Optional[float] = None,
                tensorboard_log: Optional[str] = None,
                create_eval_env: bool = False,
                policy_kwargs: Optional[Dict[str, Any]] = None,
                verbose: int = 0,
                seed: Optional[int] = None,
                device: Union[th.device, str] = "auto",
                _init_setup_model: bool = True):

        self.env = env
        self.model = sb_PPO(policy = policy,
                            env=self.env,
                            learning_rate = learning_rate,
                            n_steps = n_steps,
                            gamma = gamma,
                            batch_size = batch_size,
                            n_epochs = n_epochs,
                            gae_lambda=gae_lambda,
                            clip_range = clip_range,
                            clip_range_vf = clip_range_vf,
                            normalize_advantage=normalize_advantage,
                            ent_coef=ent_coef,
                            vf_coef=vf_coef,
                            max_grad_norm=max_grad_norm,
                            use_sde=use_sde,
                            sde_sample_freq=sde_sample_freq,
                            target_kl=target_kl,
                            tensorboard_log=tensorboard_log,
                            create_eval_env=create_eval_env,
                            policy_kwargs=policy_kwargs,
                            verbose=verbose,
                            seed=seed,
                            device=device,
                            _init_setup_model = _init_setup_model)
    
    def train_model(self, **train_params):
        self.model = self.model.learn(**train_params)
        return self.model
    
    def predict(self, environment, **test_params):
        env_test, obs_test = environment.get_env()
        """make a prediction"""
        account_memory = []
        actions_memory = []

        env_test.reset()
        for i in range(len(environment.df.index.unique())):
            action, _states = self.model.predict(obs_test, **test_params)
            obs_test, rewards, dones, info = env_test.step(action)
            if i == (len(environment.df.index.unique()) - 2):
                account_memory = env_test.env_method(method_name="save_asset_memory")
                actions_memory = env_test.env_method(method_name="save_action_memory")
            if dones[0]:
                print("hit end!")
                break

        return account_memory[0], actions_memory[0]
    
    def load_model(self, path):
        self.model = self.model.load(path)
        return self.model
    
    def save_model(self, path):
        self.model.save(path)

In [9]:
class DDPG(RLAgent):
    def __init__(self,
                policy: "MlpPolicy",
                env: None,
                learning_rate : 1e-3,
                buffer_size: 1_000_000,  # 1e6
                learning_starts: 100,
                batch_size:  100,
                tau:  0.005,
                gamma:  0.99,
                train_freq:  1,
                gradient_steps: int = -1,
                action_noise: Optional[ActionNoise] = None,
                replay_buffer_class: Optional[ReplayBuffer] = None,
                replay_buffer_kwargs: Optional[Dict[str, Any]] = None,
                optimize_memory_usage: bool = False,
                tensorboard_log: Optional[str] = None,
                create_eval_env: bool = False,
                policy_kwargs: Optional[Dict[str, Any]] = None,
                verbose: int = 0,
                seed: Optional[int] = None,
                device: Union[th.device, str] = "auto",
                _init_setup_model: bool = True):
                
        self.env = env
    
        self.model = sb_DDPG(policy = policy,
                            env=self.env,
                            learning_rate = learning_rate,
                            buffer_size = buffer_size,
                            learning_starts= learning_starts,
                            batch_size = batch_size,
                            tau = tau,
                            gamma= gamma,
                            train_freq = train_freq,
                            gradient_steps = gradient_steps,
                            action_noise= action_noise,
                            replay_buffer_class= replay_buffer_class,
                            replay_buffer_kwargs= replay_buffer_kwargs,
                            optimize_memory_usage=optimize_memory_usage,
                            tensorboard_log=tensorboard_log,
                            create_eval_env=create_eval_env,
                            policy_kwargs=policy_kwargs,
                            verbose=verbose,
                            seed=seed,
                            device=device,
                            _init_setup_model = _init_setup_model)

    def train_model(self, **train_params):
        self.model = self.model.learn(**train_params)
        return self.model

    def predict(self, environment, **test_params):
        env_test, obs_test = environment.get_env()
        """make a prediction"""
        account_memory = []
        actions_memory = []

        env_test.reset()
        for i in range(len(environment.df.index.unique())):
            action, _states = self.model.predict(obs_test, **test_params)
            obs_test, rewards, dones, info = env_test.step(action)
            if i == (len(environment.df.index.unique()) - 2):
                account_memory = env_test.env_method(method_name="save_asset_memory")
                actions_memory = env_test.env_method(method_name="save_action_memory")
            if dones[0]:
                print("hit end!")
                break

        return account_memory[0], actions_memory[0]
        
    def load_model(self, path):
        self.model = self.model.load(path)
        return self.model
    
    def save_model(self, path):
        self.model.save(path)

In [28]:
class TD3(RLAgent):
 
    def __init__(self,
                policy: "MlpPolicy",
                env: None,
                learning_rate: float =  1e-3,
                buffer_size: int = 1_000_000,  # 1e6
                learning_starts: int = 100,
                batch_size: int = 100,
                tau: float = 0.005,
                gamma: float = 0.99,
                train_freq: int = 1,
                gradient_steps: int = -1,
                action_noise: Optional[ActionNoise] = None,
                replay_buffer_class: Optional[ReplayBuffer] = None,
                replay_buffer_kwargs: Optional[Dict[str, Any]] = None,
                optimize_memory_usage: bool = False,
                tensorboard_log: Optional[str] = None,
                create_eval_env: bool = False,
                policy_kwargs: Optional[Dict[str, Any]] = None,
                verbose: int = 0,
                seed: Optional[int] = None,
                device: Union[th.device, str] = "auto",
                _init_setup_model: bool = True):
                
        self.env = env
    
        self.model = sb_TD3(policy = policy,
                            env=self.env,
                            learning_rate = learning_rate,
                            buffer_size = buffer_size,
                            learning_starts= learning_starts,
                            batch_size = batch_size,
                            tau = tau,
                            gamma= gamma,
                            train_freq = train_freq,
                            gradient_steps = gradient_steps,
                            action_noise= action_noise,
                            replay_buffer_class= replay_buffer_class,
                            replay_buffer_kwargs= replay_buffer_kwargs,
                            optimize_memory_usage=optimize_memory_usage,
                            tensorboard_log=tensorboard_log,
                            create_eval_env=create_eval_env,
                            policy_kwargs=policy_kwargs,
                            verbose=verbose,
                            seed=seed,
                            device=device,
                            _init_setup_model = _init_setup_model)

    def train_model(self, **train_params):
        self.model = self.model.learn(**train_params)
        return self.model

    def predict(self, environment, **test_params):
        env_test, obs_test = environment.get_env()
        """make a prediction"""
        account_memory = []
        actions_memory = []

        env_test.reset()
        for i in range(len(environment.df.index.unique())):
            action, _states = self.model.predict(obs_test, **test_params)
            obs_test, rewards, dones, info = env_test.step(action)
            if i == (len(environment.df.index.unique()) - 2):
                account_memory = env_test.env_method(method_name="save_asset_memory")
                actions_memory = env_test.env_method(method_name="save_action_memory")
            if dones[0]:
                print("hit end!")
                break

        return account_memory[0], actions_memory[0]
        
    def load_model(self, path):
        self.model = self.model.load(path)
        return self.model
    
    def save_model(self, path):
        self.model.save(path)

In [6]:
#Gather user parameters
with open("../user_params.yaml", "r") as stream:
    try:
        user_params = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

In [7]:
tickers = user_params["tickers"]
env_kwargs = user_params["env_params"]
train_params = user_params["train_params"]
policy_params = user_params["policy_params"]
test_params = user_params["test_params"]

In [8]:
print("\nTest 3: Downloading from Yahoo.........")
downloaded_df = DataDownloader(start_date='2009-01-01',
                                end_date='2021-10-31',
                                ticker_list= tickers).download_from_yahoo()
print(downloaded_df.head())


Test 3: Downloading from Yahoo.........
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (12924, 8)
        date       open       high        low      close     volume   tic  day
0 2008-12-31   3.070357   3.133571   3.047857   2.606277  607541200  AAPL    2
1 2008-12-31  41.590000  43.049999  41.500000  32.005894    5443100    BA    2
2 2008-12-31  43.700001  45.099998  43.700001  30.628819    6277400   CAT    2
3 2008-12-31  72.900002  74.629997  72.900002  43.314430    9964300   CVX    2
4 2009-01-02   3.067143   3.251429   3.041429   2.771173  746015200  AAPL    4


In [9]:
 # PREPROCESS DATA
print("\nTest 4: Feature engineer.........")
data_processor = DefaultFeatureEngineer(use_default=False,
                                        tech_indicator_list=env_kwargs["tech_indicator_list"],
                                        use_vix=True,
                                        use_turbulence=True,
                                        use_covar=True)
# included technical indicators as features
df_processed = data_processor.extend_data(downloaded_df)


Test 4: Feature engineer.........
Successfully added technical indicators
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (3231, 8)
Successfully added vix
Successfully added turbulence index
Successfully added covariances
Successfully added returns


In [10]:
# split data to train and test
splitter = TimeSeriesSplitter()
train = splitter.get_split_data(df_processed, '2009-01-01', '2020-06-30')
trade = splitter.get_split_data(df_processed, '2020-07-01', '2021-09-02')

In [11]:
train

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence,cov_list,return_list
0,2010-12-31,AAPL,11.533929,11.552857,11.475357,9.849803,193508000.0,4.0,0.091907,9.973602,9.686132,58.972962,56.041270,4.076954,9.738818,9.566211,17.750000,0.440064,"[[0.0002841365267912978, 0.0001814055369302911...",tic AAPL BA CAT ...
0,2010-12-31,BA,64.900002,65.290001,64.620003,52.124001,2137400.0,4.0,-0.163485,53.158962,50.459635,49.174033,30.866222,6.670857,51.751258,53.392654,17.750000,0.440064,"[[0.0002841365267912978, 0.0001814055369302911...",tic AAPL BA CAT ...
0,2010-12-31,CAT,93.830002,93.900002,93.309998,68.765228,2542700.0,4.0,1.742872,70.643151,64.949732,68.266034,72.930122,45.749416,65.923374,62.360929,17.750000,0.440064,"[[0.0002841365267912978, 0.0001814055369302911...",tic AAPL BA CAT ...
0,2010-12-31,CVX,91.580002,91.800003,91.000000,57.546162,5152900.0,4.0,1.203227,58.549676,53.227388,65.105843,107.259584,53.406336,54.660566,53.596118,17.750000,0.440064,"[[0.0002841365267912978, 0.0001814055369302911...",tic AAPL BA CAT ...
1,2011-01-03,AAPL,11.630000,11.795000,11.601429,10.063863,445138400.0,0.0,0.101104,10.012173,9.684601,62.861891,142.930195,25.488753,9.760335,9.586746,17.610001,2.341087,"[[0.0002849623588661953, 0.0001806953880267201...",tic AAPL BA CAT ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2387,2020-06-26,CVX,88.779999,88.830002,86.180000,78.338341,13766000.0,4.0,-0.752702,93.112449,76.463883,46.927254,-141.407496,18.869071,84.164340,80.878847,34.730000,1.877639,"[[0.0006521344005732147, 0.0006959985898670679...",tic AAPL BA CAT ...
2388,2020-06-29,AAPL,88.312500,90.542503,87.820000,89.329292,130646000.0,0.0,3.088530,92.294057,78.598773,62.399297,92.968100,23.584901,83.104047,76.785887,31.780001,12.311257,"[[0.000653226270088019, 0.0007078387971317241,...",tic AAPL BA CAT ...
2388,2020-06-29,BA,181.000000,194.500000,176.270004,194.490005,78499900.0,0.0,6.740439,223.696927,154.242075,53.814440,45.363542,14.268369,173.214335,154.108667,31.780001,12.311257,"[[0.000653226270088019, 0.0007078387971317241,...",tic AAPL BA CAT ...
2388,2020-06-29,CAT,123.720001,126.040001,123.279999,119.876381,2798700.0,0.0,1.323071,130.677980,113.093652,52.455047,13.227486,5.712842,118.600580,113.334042,31.780001,12.311257,"[[0.000653226270088019, 0.0007078387971317241,...",tic AAPL BA CAT ...


In [12]:
train_x, train_y = data_processor.prepare_ml_data(train)

In [13]:
train_y

array([[0.0217324 ],
       [0.01746846],
       [0.00523133],
       ...,
       [0.1439916 ],
       [0.02336786],
       [0.01399499]])

In [20]:
rf = RFAgent(**policy_params["RF_PARAMS"])

In [29]:
rf.train_model(train_x, train_y, **train_params["RF_PARAMS"])

Model trained succesfully


RandomForestRegressor(ccp_alpha=0, max_features=1, min_impurity_decrease=0,
                      min_weight_fraction_leaf=0)

In [30]:
portfolio, portfolio_cumprod, meta_coefficient = rf.predict(trade, **test_params["RF_PARAMS"])

In [31]:
portfolio

,date,account_value
0,2020-07-01,1000000
1,2020-07-02,1002717.337037
2,2020-07-06,1042091.814716
3,2020-07-07,992014.183768
4,2020-07-08,998669.002421
...,...,...
291,2021-08-26,1226212.898564
292,2021-08-27,1235442.545717
293,2021-08-30,1227431.923835
294,2021-08-31,1224065.036465


In [23]:
rf.save_model("../AgentLayer/ConventionalAgents/rf_model")

Model saved succesfully.


In [24]:
rf_loaded = rf.load_model("../AgentLayer/ConventionalAgents/rf_model")

Model loaded succesfully.


In [25]:
print(rf_loaded)

RandomForestRegressor(ccp_alpha=0, max_features=1, min_impurity_decrease=0,
                      min_weight_fraction_leaf=0)
